# 10. RNN

## 10.1 시퀀스 데이터 이해하기

- 소리, 문자열, 주가 등의 데이터를 시퀀스(sequence) 데이터로 분류한다.
- 시계열(time-series) 데이터는 시간 순서에 따라 나열된 데이터로 시퀀스 데이터에 속한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=11_hxbMiQshqFrwR5nfHxQmbn4szbc07x' width=300/>
(출처: https://priorprobability.com)

- 시퀀스 데이터는 독립동등분포(i.i.d.) 가정을 잘 위배하기 때문에 순서를 바꾸거나 과거 정보에 손실이 발생하면 데이터의 확률분포도 바뀌게 된다.
  - ex) "개가 사람을 물었다" != "사람이 개를 물었다"
- 과거 정보 또는 앞뒤 맥락 없이 미래를 예측하거나 문장을 완성하는 것은 불가능하다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1atnrWSsr9nlI2bBMR7o9MJEDmsjWWxxl' width=300/>
(출처: https://priorprobability.com)

- 시퀀스 데이터를 학습할 때는 순차적으로 들어오는 정보들을 어떻게 반영할 지를 고민해봐야 한다.

<br>

## 10.2 시퀀스 데이터 다루기

### 10.2.1 조건부확률 이용

- 이전 시퀀스의 정보를 가지고 앞으로 발생할 데이터의 확률분포를 다루기 위해 조건부확률을 이용할 수 있다.
- 이전에 배운 베이즈 법칙을 사용한다.
  - $P(A \cap B) = P(B|A)P(A)$

- ($X_1$ 부터 $X_{t}$ 까지의 결합확률분포) = ($X_1$ 부터 $X_{t-1}$ 까지의 정보가 주어졌을 때 $X_t$ 에 대한 조건부 확률분포) x ($X_1$ 부터 $X_{t-1}$ 까지의 결합확률분포)

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1fNX23CND-lpxOZ2DZr4suW8k_m8yMXQs' width=600/>

- 이를 반복하여 분해(decomposition) 하면 맨 마지막 수식처럼 각 시점(1~t)에 대해 $X_1$ 부터 $X_{s-1}$ 까지의 조건부를 가지고 $X_s$ 를 추론하는 조건부확률의 곱으로 표현할 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1yDanqp2OfWTAfl47chNfK9VRdkEuMOyY' width=600/>

- $X_1$ 부터 $X_{t-1}$ 까지의 정보가 조건부로 주어진 상황에서 현재 정보 $X_t$ 를 모델링하는 조건부확률분포를 모델링하는 것이 시퀀스 데이터를 다루는 기본적인 방법론이다.
- 아래 조건부확률은 과거의 모든 정보를 사용하지만 시퀀스 데이터를 분석할 때 **모든 과거 정보들이 필요한 것은 아니다.**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1KzW968WsycpLewkV_Ac3Rjr7ibcI6aNS' width=400/>

<br>

### 10.2.2 가변적인 데이터를 다룰 수 있는 모델

- 시퀀스 데이터를 다루기 위해선 **길이가 가변적인 데이터를 다룰 수 있는 모델이 필요**하다.
  - 조건부에 들어가는 데이터 길이는 가변적이다.
  - $X_t$ : 1 부터 t-1 까지 t-1 개의 정보를 다룰 수 있는 모델 필요
  - $X_{t+1}$ : 1 부터 t 까지 t 개의 정보를 다룰 수 있는 모델 필요

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1LG-TiAxGyZqCWGsxmEvDOOjjOATMnuMo' width=500/>

<br>

#### 10.2.2.1 자귀회귀모델 (AR, Autoregressive Model)

- 고정된 길이 $\tau$ 만큼의 시퀀스만 사용하는 경우 $AR(\tau)$ (Autoregressive Model) 자귀회귀모델이라고 부른다.
  - 초기 시점부터의 모든 정보 사용 x
  - 고정된 길이 $\tau$ 만큼의 시퀀스만 사용
- 이 모델은 $\tau$ 를 모델링 하기 전에 사전에 정해줘야 한다. (하이퍼파라미터)
- 그래서 $\tau$ 를 정하는 것도 사전 지식이 필요한 경우도 있다.
- 또 문제에 따라서는 $\tau$ 가 바뀔수도 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1hGQzVMJEOX38v-r-jz4Z9a4hwx9YnkVN' width=500/>

<br>

#### 10.2.2.2 잠재 AR 모델

- 자귀회귀모델의 $\tau$ 를 정하는 데 발생하는 이슈들을 보완할 수 있는 모델이 잠재 AR 모델이다.
- 잠재 AR 모델은 바로 이전 정보를 제외한 나머지 정보들을 $H_t$ 라는 잠재변수로 인코딩해서 활용하는 모델이다.
- 이렇게 하면 바로 이전 정보와 잠재변수 2가지 데이터만을 가지고 미래 시점을 예측할 수 있다. (길이가 고정됨)
- 이 모델의 장점은 모든 데이터를 사용하여 예측을 할 수 있고 가변적인 데이터 문제를 고정된 길이의 데이터 문제로 바꿀 수 있다는 점이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zyGXtDy6FgckEP1d6c4M06s_gB6UTTn6' width=600/>

- 하지만 과거의 정보를 어떻게 잠재변수로 인코딩 할 지에 대한 문제가 남아있다.

<br>

#### 10.2.2.3 RNN

- 잠재 AR 모델의 잠재변수 인코딩 방법 문제 해결을 위해 등장한 모델이 RNN 이다.
- 잠재변수 $H_t$ 를 Neural Network 를 통해서 과거 바로 이전의 정보$X_{t-1}$ 와 이전 잠재변수 $H_{t-1}$ 를 가지고 예측하는 모델이 RNN 이다.
- 잠재변수 $H_t$ 를 신경망을 통해 반복해서 사용하여 시퀀스 데이터의 패턴을 학습하는 모델이 RNN 이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1lBXf99LzmyXizVH4bsvEJhdChOd3QLzx' width=600/>

<br>

## 10.3 Recurrent Neural Network 을 이해하기

### 10.3.1 MLP

- 가장 기본적인 RNN 모형은 MLP 와 유사한 모형이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1vTFTLZMNBKJ0vZKsyAS_a9c1UKFs2w5M' width=400/>

- $W^{(1)}$, $W^{(2)}$ 은 시퀀스 데이터와 상관없이 불변인 가중치 행렬이다.

- 이 MLP 모델은 과거의 정보를 다룰 수 없다.
  - 입력으로 현재 시점의 데이터 $X_t$ 만 들어오기 때문

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1eJhW1s1LuQQYbbHyJEL7h1oC3YmBGJLQ' width=600/>

<br>

### 10.3.2 과거의 정보 활용

- RNN은 이전 순서의 잠재변수와 현재의 입력을 활용하여 모델링한다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1fNrvykkL75La4vkK-pkaU6RjhAMFmTLI' width=700/>

- $H_t$ 를 표현할 때 두 가지 가중치 행렬이 등장한다.
  - $W_{X}^{(1)}$ : 입력으로부터 정보를 전달하는 가중치 행렬
  - $W_{H}^{(1)}$ : 이전 잠재변수로부터 정보를 전달하는 가중치 행렬 (새로운 가중치 행렬)

- t번째 잠재변수 $H_t$ 는 현재 들어온 입력 벡터 $X_t$ 와 이전 시점의 잠재변수인 $H_{t-1}$ 를 입력받아서 계산된다.
- 이 $H_t$ 를 받아서 출력 $O_t$ 를 만들게 된다.

- 또한 $H_t$ 는 다음 시점 t+1 의 입력 잠재변수로 사용된다.
  - 감재변수인 $H_t$ 를 복제해서 다음 순서의 잠재변수를 인코딩하는 데 사용한다.


- 최종적으로 가중치 행렬은 3개가 사용된다.
  - layer 1 - $W_{X}^{(1)}$
  - layer 1 - $W_{H}^{(1)}$
  - layer 2 - $W^{(2)}$ : 잠재변수를 출력으로 만들어주는 가중치 행렬
- **이 3개의 가중치 행렬은 $t$ 에 따라서 변하지 않는다.**
  - t에 따라서 변하는 것은 잠재변수 $H$ 와 입력 벡터 $X$ 이다.
- 3개의 가중치 행렬들은 각각의 t 시점에서 동일하게 사용된다.



<br>

### 10.3.3 RNN 의 역전파

- RNN의 역전파는 **잠재변수의 연결그래프에 따라 순차적으로 계산**한다.
  - 맨 마지막 시점의 잠재변수의 그레디언트부터 처음 시점까지 차례로 흐르게 된다.
- 이를 Backpropagation Through Time(BPTT) 이라 하며 RNN의 역전파 방법이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1zJjemg2tccTJjAE9Npv43cNGPVz6z8rn' width=700/>

- 위 그림에서 빨간색 화살표로 연결된 것들이 그레디언트의 전달 경로이다.

- 잠재변수에 들어오는 그래디언트는 2개 이다.
  - 다음 시점의 잠재변수로부터 들어오는 그래디언트 벡터
  - 현재 시점의 출력에서 들어오는 그래디언트 벡터
- 들어온 그래디언트 벡터를 현재 시점의 입력과 이전 시점의 잠재변수로 전달한다.

<br>

## 10.3 BPTT 를 좀 더 살펴보자

- BPTT 를 통해 RNN의 가중치행렬의 미분을 계산해보면 아래와 같이 미분의 곱으로 이루어진 항이 계산된다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1A9ApubDRWpv3zFlC6EmRrvu2mb15PyEe' width=800/>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=17Vgvx-qanrrUnR5m3D8t4UzDgyi-VrGJ' width=800/>


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=16arEJKBAZ_P4bNhz5QiqSaNq2z0NhFFg' width=600/>

- BPTT 를 통해서 각 가중치 행렬의 미분을 계산했을 때 최종적으로 나오는 product term 은 시퀀스의 길이가 길어질수록 불안정해진다.
  - product term : i+1 부터 t 시점까지의 모든 잠재변수에 대한 미분 값의 곱
- 일반적인 BPTT를 모든 시점에 적용하게 되면 RNN 의 학습이 매우 불안정해진다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1xeaBeih8HeO84XwY6-EG647mkQ76s1Vr' width=800/>

<br>

## 10.4 기울기 소실의 해결책

- 기울기 소실
  - 과거 시점의 가중치가 0으로 줄어드는 현상
  - 과거 정보를 무시하게 될 확률이 높아진다.
- 시퀀스 길이가 길어지는 경우 BPTT 를 통한 역전파 알고리즘의 계산이 불안정해지므로 길이를 끊는 것이 필요하다.

- 이를 trunated BPTT 라 부른다.
- 이 방법을 통해 gradient vanishing 현상을 줄일 수 있다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1SDpO97uKvJRbkfsOouU9h9rJWdDigk3C' width=400/>


- 이런 문제들 때문에 Vanilla RNN 은 길이가 긴 시퀀스를 처리하는 데 문제가 있다.
- 이를 해결하기 위해 등장한 RNN 네트워크가 LSTM 과 GRU 이다.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img src='https://drive.google.com/uc?id=1Kdl_3a-yX5bidkVFS5fI1n7qasMOJp1O' width=600/>